In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import datetime
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("..") # Adds higher directory to python modules path
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
import sqlite3 as sq
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

Monthly Unemployment:  

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "LAUS" dataset, monthly  
+ Select the following dimensions: Regions, Year/Month   
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" (not available for places)   
+ Under "Year" choose your range (most recently I've selected 1990 - recent just to get it all)   
+ Select the value Labor Force (Seasonally Adjusted), Employed (Seasonally Adjusted)" , and Unemployed (Seasonally Adjusted). Unemployment rate is available but we're calculating it here in case we ever want to aggregate anything before calculating a percentage. It's been checked that this is correct and aligns with JobsEQ's data.   
+ Swap rows & columns if needed so "Regions" data is at the top, and "Year/Month" is at the side.  

When you've downloaded this csv, open it and delete the top line, make "Time" a double header, unmerge the regions and make sure they're copy-pasted across the relevant columns and keep the headers for different datapoints. Delete the data notes at the bottom, and select all of the numerical data and make them "General" format. Save as: 'JobsEQ_LAUSMonthly_GeosExceptPlaces.csv' 

In [2]:
#import monthly employment data and examine, use dtype str to import dates correctly
data = pd.read_csv('../../Data Downloads/JobsEQ_LAUSMonthly_GeosExceptPlaces.csv', dtype = str)
data.head(3)

,Time,USA,USA.1,USA.2,Tennessee (47),Tennessee (47).1,Tennessee (47).2,GNRC,GNRC.1,GNRC.2,MPO,MPO.1,MPO.2,GNRC Region,GNRC Region.1,GNRC Region.2,"Cheatham County, Tennessee (47021)","Cheatham County, Tennessee (47021).1","Cheatham County, Tennessee (47021).2","Davidson County, Tennessee (47037)","Davidson County, Tennessee (47037).1","Davidson County, Tennessee (47037).2","Dickson County, Tennessee (47043)","Dickson County, Tennessee (47043).1","Dickson County, Tennessee (47043).2","Houston County, Tennessee (47083)","Houston County, Tennessee (47083).1","Houston County, Tennessee (47083).2","Humphreys County, Tennessee (47085)","Humphreys County, Tennessee (47085).1","Humphreys County, Tennessee (47085).2","Maury County, Tennessee (47119)","Maury County, Tennessee (47119).1","Maury County, Tennessee (47119).2","Montgomery County, Tennessee (47125)","Montgomery County, Tennessee (47125).1","Montgomery County, Tennessee (47125).2","Robertson County, Tennessee (47147)","Robertson County, Tennessee (47147).1","Robertson County, Tennessee (47147).2","Rutherford County, Tennessee (47149)","Rutherford County, Tennessee (47149).1","Rutherford County, Tennessee (47149).2","Stewart County, Tennessee (47161)","Stewart County, Tennessee (47161).1","Stewart County, Tennessee (47161).2","Sumner County, Tennessee (47165)","Sumner County, Tennessee (47165).1","Sumner County, Tennessee (47165).2","Trousdale County, Tennessee (47169)","Trousdale County, Tennessee (47169).1","Trousdale County, Tennessee (47169).2","Williamson County, Tennessee (47187)","Williamson County, Tennessee (47187).1","Williamson County, Tennessee (47187).2","Wilson County, Tennessee (47189)","Wilson County, Tennessee (47189).1","Wilson County, Tennessee (47189).2"
0,Time,Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj)
1,Jan-90,126031739.2,118989531.3,7090652.576,2372397.196,2250316.966,123712.9483,576897.3031,553442.2064,23895.56508,516149.6169,496047.6945,20529.03826,604067.3046,579179.4386,25404.98597,14005.39748,13480.89802,487.2134734,273510.0775,263855.8985,9786.092357,17094.03025,16059.74623,1034.329373,2877.334714,2639.853101,240.1968585,7272.761178,6589.909032,678.8619745,27169.74475,25741.15692,1494.664552,39632.94923,37795.02973,1840.627689,20942.47349,19891.99133,1065.834469,63022.07183,60766.19301,2322.896215,3996.059609,3685.46785,307.8618946,53750.87635,51258.23763,2556.357126,3064.482218,2893.403327,175.3975759,42269.18408,41049.3645,1217.857423,35908.26686,33886.72155,2040.532431
2,Feb-90,126183221,119209184.8,7064887.393,2377326.823,2255716.413,124187.4086,578670.8943,555309.4537,24076.73747,517646.6843,497753.8713,20558.05084,605897.3518,581103.1765,25597.76796,14040.51201,13524.57162,489.8019166,274201.2632,264634.2052,9813.071441,17123.02412,16111.06616,1026.986506,2907.118983,2649.73

In [3]:
#set index to the extraneous "Time" header and transpose, then rename to "Datapoint" and reset index without dropping
data = data.set_index('Time').transpose()
data = data.rename(columns = {'Time': 'Datapoint'}).reset_index(drop = False)

In [4]:
#make sure there's no head or tail on the strings
data['index'] = data['index'].str.strip()
data['Datapoint'] = data['Datapoint'].str.strip()

In [5]:
#split and expand the geography names on the "." (where there were multiple from duplicates because of how we imported it) and take what was before as the name
boop = data['index'].str.split(pat = ".", expand = True)
data['NAME'] = boop[0]

In [6]:
#drop the original column, set the index to NAME and transpose to rename, use custom module, then transpose back and reset index without dropping
data = data.drop(columns = 'index').set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [7]:
#go from wide to long format with .melt() naming the columns as single column "Time", keeping "Datapoint" and "NAME" as is, and calling the number "Value"
cols = data.columns
data = data.melt(var_name = 'Time', id_vars = ['NAME', 'Datapoint'], value_vars = cols, value_name = 'Value')
data.head(2)

,NAME,Datapoint,Time,Value
0,United States,Labor Force (Seas Adj),Jan-90,126031739.2
1,United States,Employed (Seas Adj),Jan-90,118989531.3


In [8]:
#set index and transpose on datapoint to rename the original categories from JobsEQ, then transpose and reset index again without dropping
data = data.set_index('Datapoint').transpose()
data = data.rename(columns = {'Labor Force (Seas Adj)': 'LF:Labor Force', 'Employed (Seas Adj)': 'LF:Employed', 
                              'Unemployed (Seas Adj)': 'LF:Unemployed'})
data = data.transpose().reset_index(drop = False)

In [9]:
#make sure the value is a float datatype
data['Value'] = data['Value'].astype(float)

In [10]:
#create a new column equal to the value
data['New Value'] = data['Value']
#use .loc to say that in instances where the datapoint is equal to unemployment rate, new value is equal to value times 100 to match percent formatting
data.loc[data['Datapoint'] == 'LF:Unemployment %', 'New Value'] = data['Value'] * 100

In [11]:
data = data.drop(columns = 'Value').rename(columns = {'New Value': 'Value'})
data.head()

,Datapoint,NAME,Time,Value
0,LF:Labor Force,United States,Jan-90,1.260317e+08
1,LF:Employed,United States,Jan-90,1.189895e+08
2,LF:Unemployed,United States,Jan-90,7.090653e+06
3,LF:Labor Force,Tennessee,Jan-90,2.372397e+06
4,LF:Employed,Tennessee,Jan-90,2.250317e+06


In [12]:
# RUN THIS IS YOU'RE INCLUDING THE YEAR 2000
weirds = ['Jan-00', 'Feb-00', 'Mar-00', 'Apr-00', 'May-00', 'Jun-00', 'Jul-00', 'Aug-00', 'Sep-00', 'Oct-00', 'Nov-00', 'Dec-00']
twothousand = data.loc[data['Time'].isin(weirds)]
data = data.loc[~data['Time'].isin(weirds)]

In [13]:
data.tail()

,Datapoint,NAME,Time,Value
22624,LF:Employed,"Williamson County, Tennessee",Jan-23,133997.583200
22625,LF:Unemployed,"Williamson County, Tennessee",Jan-23,3398.490265
22626,LF:Labor Force,"Wilson County, Tennessee",Jan-23,81564.890530
22627,LF:Employed,"Wilson County, Tennessee",Jan-23,79374.518470
22628,LF:Unemployed,"Wilson County, Tennessee",Jan-23,2138.321426


In [14]:
#format the time for datetime by splitting the string and creating a month and year column
boop = data['Time'].str.split(pat = "-", expand = True)
data['month'] = boop[0]
data['year'] = boop[1]

In [15]:
twothousand.head()

,Datapoint,NAME,Time,Value
6840,LF:Labor Force,United States,Jan-00,1.435719e+08
6841,LF:Employed,United States,Jan-00,1.375764e+08
6842,LF:Unemployed,United States,Jan-00,6.026699e+06
6843,LF:Labor Force,Tennessee,Jan-00,2.850301e+06
6844,LF:Employed,Tennessee,Jan-00,2.737877e+06


In [16]:
# RUN THIS IS YOU'RE INCLUDING THE YEAR 2000
boop = twothousand['Time'].str.split(pat = "-", expand = True)
twothousand['month'] = boop[0]
twothousand['year'] = boop[1]

In [17]:
# RUN THIS IS YOU'RE INCLUDING THE YEAR 2000
df = data.append(twothousand)
data = df

C:\Users\jmccall\AppData\Local\Temp\ipykernel_20368\2624616270.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = data.append(twothousand)


In [18]:
#set the index to the month and transpose/rename before transposing back and resetting the index without dropping
data = data.set_index('month').transpose()
data = data.rename(columns = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 
                              'May': 5, 'June': 6, 'July': 7, 'August': 8, 
                              'September': 9, 'October': 10, 'November': 11, 'December': 12, 
                             'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 
                              'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 
                              'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12})
data = data.transpose().reset_index(drop = False)

In [19]:
data['year'].unique()

array(['90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '01',
       '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '00'], dtype=object)

In [20]:
#do the same for year
data = data.set_index('year').transpose()
data = data.rename(columns = {'90': 1990, '91': 1991, '92': 1992, '93': 1993, '94': 1994, '95': 1995, 
                              '96': 1996, '97': 1997, '98': 1998, '99': 1999, 
                              '00': 2000, '1': 2001, '2': 2002, '3': 2003, '4': 2004, '5': 2005, 
                              '6': 2006, '7': 2007, '8': 2008, '9': 2009, '10': 2010, '11': 2011, 
                              '12': 2012, '13': 2013, '14': 2014, '15': 2015, '16': 2016, '17': 2017, 
                              '18': 2018, '19': 2019, '20': 2020, '21': 2021, '22': 2022})
data = data.transpose().reset_index(drop = False)

In [21]:
#add a filler for datetime to assume that this refers to the first day of each month
data['day'] = 1

In [22]:
data.head(2)

,year,month,Datapoint,NAME,Time,Value,day
0,1990,1,LF:Labor Force,United States,Jan-90,126031739.2,1
1,1990,1,LF:Employed,United States,Jan-90,118989531.3,1


In [23]:
#set these to datatype integer to input to datetime
cols = ['month', 'year', 'day']
data[cols] = data[cols].astype(int)

In [24]:
#we only want years 2000 on
data = data.loc[data['year'] > 1999]

In [25]:
#create a time stamp with these
data['TimeStamp'] = pd.to_datetime(data[['year', 'month', 'day']])

In [26]:
data.head(2)

,year,month,Datapoint,NAME,Time,Value,day,TimeStamp
12996,2010,1,LF:Labor Force,United States,Jan-10,154780469.6,1,2010-01-01
12997,2010,1,LF:Employed,United States,Jan-10,139139406.2,1,2010-01-01


In [27]:
#drop the Time and day columns and check before exporting
data = data.drop(columns = ['Time', 'day', 'year', 'month'])

In [28]:
#long to wide
data = pd.pivot(data, index = ['NAME', 'TimeStamp'], columns = 'Datapoint', values = 'Value')

In [29]:
data['LF:Unemployment %'] = (data['LF:Unemployed']/data['LF:Labor Force']) * 100

In [30]:
data.head(2)

Datapoint                              LF:Employed LF:Labor Force LF:Unemployed LF:Unemployment %
NAME                       TimeStamp                                                             
Cheatham County, Tennessee 2000-01-01   19305.6373    19845.43144    504.622151          2.542762
                           2000-02-01  19202.97258    19754.63085    519.562122          2.630078

In [31]:
#make sure the index is reset
data.reset_index(drop = False, inplace = True)
data = data.rename_axis(None, axis=1)
data.head()

,NAME,TimeStamp,LF:Employed,LF:Labor Force,LF:Unemployed,LF:Unemployment %
0,"Cheatham County, Tennessee",2000-01-01,19305.6373,19845.43144,504.622151,2.542762
1,"Cheatham County, Tennessee",2000-02-01,19202.97258,19754.63085,519.562122,2.630078
2,"Cheatham County, Tennessee",2000-03-01,19118.65875,19666.6465,524.99364,2.669462
3,"Cheatham County, Tennessee",2000-04-01,19059.10019,19594.41154,524.318691,2.675858
4,"Cheatham County, Tennessee",2000-05-01,19016.58686,19539.10814,522.217909,2.67268


In [32]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ LAUS'

In [35]:
#final check
data.head()

,NAME,TimeStamp,LF:Employed,LF:Labor Force,LF:Unemployed,LF:Unemployment %,GEO_ID,Source
0,"Cheatham County, Tennessee",2000-01-01,19305.6373,19845.43144,504.622151,2.542762,0500000US47021,JobsEQ LAUS
1,"Cheatham County, Tennessee",2000-02-01,19202.97258,19754.63085,519.562122,2.630078,0500000US47021,JobsEQ LAUS
2,"Cheatham County, Tennessee",2000-03-01,19118.65875,19666.6465,524.99364,2.669462,0500000US47021,JobsEQ LAUS
3,"Cheatham County, Tennessee",2000-04-01,19059.10019,19594.41154,524.318691,2.675858,0500000US47021,JobsEQ LAUS
4,"Cheatham County, Tennessee",2000-05-01,19016.58686,19539.10814,522.217909,2.67268,0500000US47021,JobsEQ LAUS


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   NAME               3192 non-null   object        
 1   TimeStamp          3192 non-null   datetime64[ns]
 2   LF:Employed        3192 non-null   object        
 3   LF:Labor Force     3192 non-null   object        
 4   LF:Unemployed      3192 non-null   object        
 5   LF:Unemployment %  3192 non-null   object        
 6   GEO_ID             2520 non-null   object        
 7   Source             3192 non-null   object        
dtypes: datetime64[ns](1), object(7)
memory usage: 199.6+ KB


In [37]:
#export to the SQLite database as only the JobsEQ monthly data
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JOBSEQ_LAUS_Monthly', conn, if_exists = 'replace', index = False)

3192